In [1]:
import torch
from memristor.devices import DynamicMemristorStuck
from memristor.crossbar.model import LineResistanceCrossbar
from tqdm import tqdm

In [2]:
torch.set_default_dtype(torch.float64)
crossbar_params = {'r_wl': 20, 'r_bl': 20, 'r_in': 10, 'r_out': 10, 'V_SOURCE_MODE': '|_'}
memristor_model = DynamicMemristorStuck
memristor_params = {'frequency': 1e8, 'temperature': 273 + 40}
ideal_w = torch.ones([48, 16])*65e-6
crossbar = LineResistanceCrossbar(memristor_model, memristor_params, ideal_w, crossbar_params)

/Users/jvjv/Github/pytorch-metal-oxide-memristor-crossbar/memristor/crossbar/model.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  self.fitted_w = torch.tensor([[self.memristors[i][j].g_linfit for j in range(ideal_w.shape[1])]


In [3]:
# randomize weights
n_reset = 40
t_p_reset = 0.5e-3
v_p_bl = 1.5 * torch.cat([torch.linspace(1, 1.2, 16), 1.2 * torch.ones(16, ),
                          torch.linspace(1.2, 1, 16)], dim=0)
for j in tqdm(range(n_reset)):
    crossbar.lineres_memristive_programming(torch.zeros(16, ), v_p_bl, t_p_reset, log_power=True)

v_wl_applied = 0.3*(torch.randint(low=0, high=2, size=[16,]))+0.1
print("input", v_wl_applied)
v_bl_applied = torch.zeros(48)
x = crossbar.lineres_memristive_vmm(v_wl_applied, v_bl_applied, log_power=True)
print("power:")
for ticket in crossbar.power_log:
    print(f"{ticket.name} - {ticket.op_type}")
    print("Total Power:", ticket.power_total)
    print("Memristor Power:", ticket.power_memristor)
    print("Word Line Power:", ticket.power_wordline)
    print("Bit Line Power:", ticket.power_bitline)

  0%|          | 0/40 [00:00<?, ?it/s]/Users/jvjv/Github/pytorch-metal-oxide-memristor-crossbar/memristor/crossbar/model.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.ideal_w[i,j] = torch.tensor(self.memristors[i][j].g_0)
100%|██████████| 40/40 [01:50<00:00,  2.76s/it]


input tensor([0.4000, 0.1000, 0.4000, 0.4000, 0.1000, 0.1000, 0.4000, 0.4000, 0.4000,
        0.1000, 0.4000, 0.1000, 0.4000, 0.1000, 0.1000, 0.1000])
power:
1679877535801832000 - PROGRAMMING
Total Power: 0.09715879601959591
Memristor Power: 0.04499898934732483
Word Line Power: 0.04377815729238954
Bit Line Power: 0.008381649379881553
1679877538640993000 - PROGRAMMING
Total Power: 0.0967079343688896
Memristor Power: 0.04471399630297758
Word Line Power: 0.04369813677294471
Bit Line Power: 0.008295801292967305
1679877541483797000 - PROGRAMMING
Total Power: 0.09624852226005057
Memristor Power: 0.04442258681378765
Word Line Power: 0.04361720540005339
Bit Line Power: 0.008208730046209534
1679877544363655000 - PROGRAMMING
Total Power: 0.09583323280538372
Memristor Power: 0.04416780498738782
Word Line Power: 0.04354036956675991
Bit Line Power: 0.00812505825123599
1679877547112738000 - PROGRAMMING
Total Power: 0.09549568944841065
Memristor Power: 0.043967938417787775
Word Line Power: 0.04347387